# Homework 5 Answers

This Jupyter notebook contains all of my answers to the Week 5 homework questions for the 2023 cohort of the Data Engineering Zoomcamp. For convenience, each question is restated before giving the corresponding answer; the list of questions *without* corresponding answers can be found in `01-questions.md`, which can be found in the current directory.

Please note that this notebook **assumes that the `05-batch/homework` directory is your present working directory**. 

## Set-Up

First, let's install all of the requirements we'll need to run this notebook:

In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


Note that you'll need to make sure that `spark` is installed in order for `pyspark` to work; please refer to the [instructions provided in the Data Engineering Zoomcamp GitHub repository](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/week_5_batch_processing/setup) for how to install `spark` on your particular machine.

Next, let's import the modules we need:

In [2]:
import os
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types

# See `utils.py` for descriptions of `download_csv` and `get_parquet_file_sizes`:
from utils import download_csv, get_parquet_file_sizes

Let's now download the two CSV files we need to answer the homework questions:

In [3]:
fhvhv_csv_path = download_csv(
    "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz"
)
zone_lookup_path = download_csv(
    "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"
)

We'll now start our `spark` session:

In [4]:
spark = SparkSession.builder.appName("zoomcamp_hw5").getOrCreate()

If you so desire, you can monitor the progress of the jobs we send to our `spark` cluster at the following port:

In [5]:
spark.sparkContext.uiWebUrl

'http://192.168.1.126:4040'

## Question 1

### Question

Create a `spark` session with `pyspark` and call `spark.version`. What's the output? Choose from the following options:
- 3.3.2
- 2.1.4
- 1.2.3
- 5.4 

### Answer

Executing `spark.version` shows that we're running version `3.3.2` of `pyspark`:

In [6]:
spark.version

'3.3.2'

## Question 2

### Question

Create a `pyspark` dataframe for the downloaded FHVHV CSV into `spark`, and repartition it into 12 partitions. After this, save the data to Parquet. What is the average size of the 12 created Parquet files (i.e. the files ending with a `.parquet` extension)?

### Answer

Before loading the FHVHV CSV into `spark`, we first need to work out the schema for this data. To help us do this, let's use `pandas` to load the first five rows of the dataset:

In [7]:
df = pd.read_csv(fhvhv_csv_path, nrows=5)
df

,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B02764,2021-06-01 00:02:41,2021-06-01 00:07:46,174,18,N,B02764
1,B02764,2021-06-01 00:16:16,2021-06-01 00:21:14,32,254,N,B02764
2,B02764,2021-06-01 00:27:01,2021-06-01 00:42:11,240,127,N,B02764
3,B02764,2021-06-01 00:46:08,2021-06-01 00:53:45,127,235,N,B02764
4,B02510,2021-06-01 00:45:42,2021-06-01 01:03:33,144,146,N,NaN


From this output, it appears that the FHVHV dataset has the following schema:
- `dispatching_base_num` is a string
- `pickup_datetime` is a timestamp
- `dropoff_datetime	` is a timestamp
- `PULocationID` is a (short) integer
- `SR_Flag` is a string
- `Affiliated_base_number` is a string

We can specify this schema in `pyspark` as follows:

In [8]:
schema = types.StructType(
    [
        types.StructField("dispatching_base_num", types.StringType(), True),
        types.StructField("pickup_datetime", types.TimestampType(), True),
        types.StructField("dropoff_datetime", types.TimestampType(), True),
        types.StructField("PULocationID", types.IntegerType(), True),
        types.StructField("DOLocationID", types.IntegerType(), True),
        types.StructField("SR_Flag", types.StringType(), True),
        types.StructField("Affiliated_base_number", types.StringType(), True),
    ]
)

With the schema specified, let's now load the CSV data as a `pyspark` dataframe:

In [9]:
df = spark.read.option("header", "true").schema(schema).csv(fhvhv_csv_path)

We can now repartition this dataframe into 12 separate partitions:

In [10]:
df = df.repartition(12)

Let's now save this reparatitioned table into `parquet` format; we'll store the saved `parquet` files in a subdirectory called `data`:

In [11]:
if os.path.isdir("data/"):
    os.rmtree("data/")
df.write.parquet("data/")

We'll now use the `get_parquet_file_sizes` utility function we've defined in `utils.py` to list the file sizes of all the `.parquet` files we've saved in the `data` subdirectory; note that all of the listed files sizes are in **MB**:

In [12]:
get_parquet_file_sizes(dir_to_search="data")

{'part-00002-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.678828239440918,
 'part-00001-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.666645050048828,
 'part-00011-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.66179370880127,
 'part-00005-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.67503070831299,
 'part-00004-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.67267417907715,
 'part-00000-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.674582481384277,
 'part-00010-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.666647911071777,
 'part-00009-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.675537109375,
 'part-00003-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.673087120056152,
 'part-00007-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.67320442199707,
 'part-00008-082d240a-463d-4186-95c4-3d986a7c15b9-c000.snappy.parquet': 23.671749114990

It appears that the average size of the Parquet files we've just created is around **24 MB**.

Now that we've created these Parquet files, it makes sense to use them to recreate our dataframe; this should speed-up our queries for subsequent questions: 

In [13]:
df = spark.read.parquet("data")

## Question 3

### Question

How many taxi trips started on June 15 in the FHVHV dataset you downloaded?

### Answer

This question is probably most 'cleanly' answered by using a SQL query, so let's first tell `spark` to create a table called `trips_data` from our `df` which we can query:

In [14]:
df.createOrReplaceTempView("trips_data")

We can now use the following simple SQL query to count the number of drips occurring on `2021-06-15`:

In [15]:
spark.sql(
    """
SELECT
    count(1)
FROM
    trips_data
WHERE
    DATE(pickup_datetime) = '2021-06-15';
"""
).show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



i.e. `452470` trips occurred on `2021-06-15`.

## Question 4

### Question

How much time (measured in hours) did the longest taxi trip in the dataset take?

### Answer

We can find length of the longest trip in our FHV dataframe in three steps:
1. Compute the time of each trip in seconds by subtracting the UNIX timestampe of the `dropoff_datetime` from the UNIX timestampe of the `pickup_datetime`. For those who are unaware, the UNIX timestamp of a particular time measures the number of seconds that have elapsed between X and that particular time, which means that the difference between two UNIX timestamps is simply the seconds that have elapsed between those two times.
2. Compute the time of each trip in hours by dividing the previously calculated trip time by 3600 (i.e. there are 3600 seconds in an hour).
3. Return the maximum trip time in hours.

Doing all this shows us that the longest FHVHV trip in June 2021 took around **67 hours**:

In [16]:
df.withColumn(
    "trip_time_in_secs",
    F.unix_timestamp(df.dropoff_datetime) - F.unix_timestamp(df.pickup_datetime),
).withColumn("trip_time_in_hrs", F.col("trip_time_in_secs") / 3600).select(
    F.max("trip_time_in_hrs")
).show()

+---------------------+
|max(trip_time_in_hrs)|
+---------------------+
|     66.8788888888889|
+---------------------+



## Question 5

### Question

Which port on your local machine do you use to access the Spark User Interface dashboard?

### Answer

As we previously shown near the start of this notebook, we can use `pyspark` to get a link to the Spark User Interface dashboard:

In [17]:
spark.sparkContext.uiWebUrl

'http://192.168.1.126:4040'

From this link, we can clearly see that the Spark User Interface runs on **Port 4040** of our local system.

## Question 6

### Question

With the help of the Taxi Zone Lookup table, find the name of the most frequent pickup zone location.

### Answer

Let's first create a `pyspark` dataframe for the Taxi Zone lookup data we've downloaded. To do this, we need to work out the schema of this data. Once again, we'll work out the schema to specify for this data by using `pandas` to read the first fice rows of the CSV:

In [18]:
df_zones = pd.read_csv(zone_lookup_path, nrows=5)
df_zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


From this, we see that this data uses the following schema:
- `LocationID` is a (short) integer
- `Borough` is a string
- `Zone` is a string
- `service_zone` is a string

We can specify this schema in `pyspark` like so:

In [19]:
schema = types.StructType(
    [
        types.StructField("LocationID", types.ShortType(), False),
        types.StructField("Borough", types.StringType(), False),
        types.StructField("Zone", types.StringType(), False),
        types.StructField("service_zone", types.StringType(), False),
    ]
)

With the schema specified, let's now read in this CSV to `pyspark`:

In [20]:
df_zone = spark.read.option("header", "true").schema(schema).csv(zone_lookup_path)

Instead of merging the FHV dataframe `df` and the Taxi zone dataframe `df_zone`, it's more computationally efficient (and also cleaner/easier, in my personal opinion) to answer this question with a SQL query that uses a Common Table Expression (CTE). 

To do this, we'll first need to tell `pyspark` to create a table for the `df_zone` dataframe, which we'll call `zone_lookup`:

In [21]:
df_zone.createOrReplaceTempView("zone_lookup")

We can then query both the `zone_lookup` we've just created, as well as the `trips_data` table we previously created, to answer our question. More specifically, we 

In [22]:
spark.sql(
    """
-- Count number of trips starting in each zone and keep only zone with most trips:
WITH most_freq_pu AS (
    SELECT 
        PULocationID, 
        COUNT(1) AS num_pickups
    FROM 
        trips_data
    GROUP BY 
        PULocationID
    ORDER BY 
        num_pickups DESC
    LIMIT 
        1
)
-- Find corresponding zone name:
SELECT 
    zl.Zone, pu.num_pickups
FROM
    zone_lookup AS zl,
    most_freq_pu AS pu
WHERE
    zl.LocationID = pu.PULocationID;
"""
).show()

+-------------------+-----------+
|               Zone|num_pickups|
+-------------------+-----------+
|Crown Heights North|     231279|
+-------------------+-----------+



We see here that the **Crown Heights North** zone had the most FHVHV taxi trip pick-ups in June 2021.